<img src="https://openfisca.org/img/logo-openfisca.svg" width="198" align="right">
<h1 id="tocheading">Getting Started</h1>

This notebook illustrates the [key concepts](https://openfisca.org/doc/key-concepts/index.html) of OpenFisca that you will find in the documentation.

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>


## Calculate a variable

Do some imports

In [1]:
from openfisca_core.simulation_builder import SimulationBuilder
from openfisca_france import FranceTaxBenefitSystem

libyaml is not installed in your environment. This can make OpenFisca slower to start. Once you have installed libyaml, run 'pip uninstall pyyaml && pip install pyyaml --no-cache-dir' so that it is used in your Python environment.



Initialize the french [Tax and Benefit system](https://openfisca.org/doc/key-concepts/tax_and_benefit_system.html) : it launches the entire legislation.

In [2]:
tax_benefit_system = FranceTaxBenefitSystem()

Create a [scenario](https://openfisca.org/doc/key-concepts/input_data.html#scenario): first step to give a familial situation for which you want to calculate taxes or benefits.

In [3]:
scenario = tax_benefit_system.new_scenario()

Initialize a test case (the familial situation) in the scenario:
here we have one parent and two children.

And use this test case to create a [simulation](https://openfisca.org/doc/key-concepts/simulation.html#simulation-the-framework-of-computation): the OpenFisca frame for calculating taxes or benefits.

In [4]:
TEST_CASE = {
    'individus': {
        'parent1': {
            'age': {'2015-01': 30},
            'salaire_de_base': {'2015': 20000}
        },
        'enfant1': {
            'age': {'2015-01': 12}
        },
        'enfant2': {
            'age': {'2015-01': 18}
        }
    }
}

simulation_builder = SimulationBuilder()
simulation = simulation_builder.build_from_entities(tax_benefit_system, TEST_CASE)

Calculate a variable : `af` (allocations familiales) for example.

In [5]:
simulation.calculate('af', '2015-01')

ValueError: Input [False False] is not a valid value for the entity individu (size = 2 != 3 = count)

## Test the impact of a reform

A reform represents a modified version of the real Tax and Benefit legislation* 

Reforms are gather in the module [`Reform`](https://github.com/openfisca/openfisca-france/tree/master/openfisca_france/reforms) of OpenFisca.

You have then to import the module extension corresponding to the reform of your concern.

Here we choose the fiscal reform of Landais, Piketty and Saez on the income tax (http://www.revolution-fiscale.fr/la-reforme-proposee). 

It is already implemented in OpenFisca under the name `landais_piketty_saez` .

In [ ]:
from openfisca_france.reforms import landais_piketty_saez

Create a modified version of the tax and benefit system, affected by the changes introduced by the reform

In [ ]:
reform = landais_piketty_saez.landais_piketty_saez(tax_benefit_system)

Create the scenario you want to test.

In [ ]:
def init_profile(scenario):
    scenario.init_single_entity(
        period = '2013',
        parent1 = dict(
            age = 40,
            salaire_de_base = 50000,
            ),
        )
    return scenario

With the reform

In [ ]:
# Indicate that you want to perfom the reform on this scenario
reform_scenario = init_profile(reform.new_scenario())

#Simulate the reform
reform_simulation = reform_scenario.new_simulation() 

# Choose the variable you want to calcul : here the disposable income, "revenu_disponible"
reform_simulation.calculate('revenu_disponible', '2013')

Without the reform: the baseline (counterfactual)

In [ ]:
# Indicate that you want to perfom the standard system on this scenario
baseline_scenario = init_profile(tax_benefit_system.new_scenario())

# Simulate the standard scenario
baseline_simulation = baseline_scenario.new_simulation()

# Choose the variable you want to calcul
baseline_simulation.calculate('revenu_disponible', '2013')

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')